In [1]:
import numpy as np
import pandas as pd
import os,sys

In [2]:
def effDictFromFile(fileName,yLabel):
    
    fileDict = {'mass (GeV)' : [], 'lifetime (ns)' : [], 'width (GeV)' : []}
    fileDict[yLabel] = []
    
    # Split file by lifetime:
    with open(fileName,'r') as f:
        blocks = f.read().split('#: Lifetime')[1:]
        for b in blocks:
            b = b.split('\n')
            lifetime = b[0].split(',')[1].replace('ns','')
            if lifetime.strip().lower() == 'stable':
                lifetime = np.inf
                width = 0.0
            else:
                lifetime = eval(lifetime)
                width = 6.582e-25/(lifetime*1e-9)
            for row in b[2:]:
                row = row.strip()
                if not row:
                    continue
                if row[0] == "#":
                    continue
                mass,yval = row.split(',')
                try:
                    yval = eval(yval)
                except:
                    yval = 0.0
                mass = eval(mass)
                fileDict['mass (GeV)'].append(mass)
                fileDict['lifetime (ns)'].append(lifetime)
                fileDict['width (GeV)'].append(width)
                fileDict[yLabel].append(yval)   
    
    return fileDict

def limitDictFromFile(inputFile):
    
    if 'Chargino' in inputFile:
        model='Chargino'
    elif 'Stau' in inputFile:
        model='Stau'
    
    lifetime = inputFile.split(model)[1].split('.csv')[0]
    lifetime = lifetime.replace('ns','')
    if lifetime.strip().lower() == 'stable':
        lifetime = np.inf
        width = 0.0
    else:
        lifetime = eval(lifetime)
        width = 6.582e-25/(lifetime*1e-9)

    fileDict = {'mass (GeV)' : [], 'lifetime (ns)' : [], 'width (GeV)' : [], 
                'Expected Limit (pb)' : [], 'Observed Limit (pb)' : [], 'Signal Cross-Section (pb)' : []}

    # Split file by lifetime:
    with open(inputFile,'r') as f:
        if model == 'Chargino':
            blocks = f.read().split('Chargino Mass [TeV]')[1:]
        elif model == 'Stau':
            blocks = f.read().split('Stau Mass [TeV]')[1:]
        for b in blocks:
            b = b.split('\n')
            # Select correct label corresponding to block
            for yLabel in fileDict:
                if yLabel[:10].lower() in b[0].lower():
                    break
            for row in b[1:]:
                row = row.strip()
                if not row:
                    continue
                if row[0] == "#":
                    continue
                mass,yval = row.split(',')[:2]
                try:
                    yval = eval(yval)
                except:
                    yval = 0.0
                mass = eval(mass)*1000.0
                if mass not in fileDict['mass (GeV)']:
                    fileDict['mass (GeV)'].append(mass)
                    fileDict['lifetime (ns)'].append(lifetime)
                    fileDict['width (GeV)'].append(width)
                fileDict[yLabel].append(yval)   
                
    return fileDict

### Acceptance info

In [3]:
# inputFile = './HEPData-ins2080541-v1-csv/Acceptance,Chargino.csv'
inputFile = './HEPData-ins2080541-v1-csv/Acceptance,Stau.csv'
# Create dict to store information:
accDict = effDictFromFile(inputFile,yLabel = 'acceptance')
accDF = pd.DataFrame(accDict)            
accDF

,mass (GeV),lifetime (ns),width (GeV),acceptance
0,100.0,0.3,2.194000e-15,0.040
1,200.0,0.3,2.194000e-15,0.036
2,300.0,0.3,2.194000e-15,0.033
3,400.0,0.3,2.194000e-15,0.025
4,450.0,0.3,2.194000e-15,0.000
...,...,...,...,...
73,650.0,inf,0.000000e+00,0.960
74,700.0,inf,0.000000e+00,0.962
75,800.0,inf,0.000000e+00,0.974
76,900.0,inf,0.000000e+00,0.978


### Event-level efficiency

In [4]:
# inputFile = './HEPData-ins2080541-v1-csv/Event-levelefficiency,Chargino.csv'
inputFile = './HEPData-ins2080541-v1-csv/Event-levelefficiency,Stau.csv'
evteffDict = effDictFromFile(inputFile,yLabel = 'event eff')
# Define data frame
evteffDF = pd.DataFrame(evteffDict)            
evteffDF

,mass (GeV),lifetime (ns),width (GeV),event eff
0,100.0,0.3,2.194000e-15,0.000
1,200.0,0.3,2.194000e-15,0.001
2,300.0,0.3,2.194000e-15,0.001
3,400.0,0.3,2.194000e-15,0.003
4,450.0,0.3,2.194000e-15,0.000
...,...,...,...,...
73,650.0,inf,0.000000e+00,0.017
74,700.0,inf,0.000000e+00,0.021
75,800.0,inf,0.000000e+00,0.027
76,900.0,inf,0.000000e+00,0.033


### SR-Inclusive Low efficiency

In [5]:
# inputFile = './HEPData-ins2080541-v1-csv/Efficiency,SR-Inclusive_Low,Chargino.csv'
inputFile = './HEPData-ins2080541-v1-csv/Efficiency,SR-Inclusive_Low,Stau.csv'
srlowDict = effDictFromFile(inputFile,yLabel = 'SR-Inclusive_Low eff')
# Define data frame
srlowDF = pd.DataFrame(srlowDict)            
srlowDF

,mass (GeV),lifetime (ns),width (GeV),SR-Inclusive_Low eff
0,100.0,0.3,2.194000e-15,0.000
1,200.0,0.3,2.194000e-15,0.001
2,300.0,0.3,2.194000e-15,0.010
3,400.0,0.3,2.194000e-15,0.012
4,450.0,0.3,2.194000e-15,0.000
...,...,...,...,...
73,650.0,inf,0.000000e+00,0.020
74,700.0,inf,0.000000e+00,0.022
75,800.0,inf,0.000000e+00,0.027
76,900.0,inf,0.000000e+00,0.034


### SR-Inclusive Low efficiency

In [6]:
# inputFile = './HEPData-ins2080541-v1-csv/Efficiency,SR-Inclusve_High,Chargino.csv'
inputFile = './HEPData-ins2080541-v1-csv/Efficiency,SR-Inclusve_High,Stau.csv'
srhighDict = effDictFromFile(inputFile,yLabel = 'SR-Inclusive_High eff')
# Define data frame
srhighDF = pd.DataFrame(srhighDict)            
srhighDF

,mass (GeV),lifetime (ns),width (GeV),SR-Inclusive_High eff
0,100.0,0.3,2.194000e-15,0.000
1,200.0,0.3,2.194000e-15,0.001
2,300.0,0.3,2.194000e-15,0.001
3,400.0,0.3,2.194000e-15,0.003
4,450.0,0.3,2.194000e-15,0.000
...,...,...,...,...
73,650.0,inf,0.000000e+00,0.017
74,700.0,inf,0.000000e+00,0.021
75,800.0,inf,0.000000e+00,0.027
76,900.0,inf,0.000000e+00,0.033


### Cross-section limits

In [7]:
allDicts = []
# inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Chargino10ns.csv'
# allDicts.append(limitDictFromFile(inputFile))

# inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Chargino30ns.csv'
# allDicts.append(limitDictFromFile(inputFile))

# inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Chargino4ns.csv'
# allDicts.append(limitDictFromFile(inputFile))

# inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,CharginoStable.csv'
# allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Stau1ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Stau10ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Stau30ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,Stau3ns.csv'
allDicts.append(limitDictFromFile(inputFile))

inputFile = './HEPData-ins2080541-v1-csv/CrossSectionLimit,StauStable.csv'
allDicts.append(limitDictFromFile(inputFile))


xsecAll = {}
for d in allDicts:
    for k,vals in d.items():
        if not k in xsecAll:
            xsecAll[k] = []
        xsecAll[k] += vals

In [8]:
xsecDF = pd.DataFrame(xsecAll)
xsecDF

,mass (GeV),lifetime (ns),width (GeV),Expected Limit (pb),Observed Limit (pb),Signal Cross-Section (pb)
0,450.0,1.0,6.582000e-16,0.006030,0.010000,0.001100
1,550.0,1.0,6.582000e-16,0.003780,0.004770,0.000426
2,600.0,1.0,6.582000e-16,0.003250,0.004320,0.000276
3,100.0,10.0,6.582000e-17,8.760000,13.900000,0.366000
4,200.0,10.0,6.582000e-17,0.026100,0.033200,0.030300
5,300.0,10.0,6.582000e-17,0.003400,0.003780,0.006250
6,400.0,10.0,6.582000e-17,0.001320,0.002310,0.001860
7,450.0,10.0,6.582000e-17,0.000933,0.001660,0.001100
8,500.0,10.0,6.582000e-17,0.000830,0.001220,0.000674
9,550.0,10.0,6.582000e-17,0.000592,0.000779,0.000426


In [9]:
allDFs = [accDF,evteffDF,srlowDF,srhighDF,xsecDF]

df = accDF
for dd in allDFs[1:]:
    df = df.merge(dd,on=['mass (GeV)','lifetime (ns)','width (GeV)'],how='outer')

In [10]:
df

,mass (GeV),lifetime (ns),width (GeV),acceptance,event eff,SR-Inclusive_Low eff,SR-Inclusive_High eff,Expected Limit (pb),Observed Limit (pb),Signal Cross-Section (pb)
0,100.0,0.3,2.194000e-15,0.040,0.000,0.000,0.000,NaN,NaN,NaN
1,200.0,0.3,2.194000e-15,0.036,0.001,0.001,0.001,NaN,NaN,NaN
2,300.0,0.3,2.194000e-15,0.033,0.001,0.010,0.001,NaN,NaN,NaN
3,400.0,0.3,2.194000e-15,0.025,0.003,0.012,0.003,NaN,NaN,NaN
4,450.0,0.3,2.194000e-15,0.000,0.000,0.000,0.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
73,650.0,inf,0.000000e+00,0.960,0.017,0.020,0.017,NaN,NaN,NaN
74,700.0,inf,0.000000e+00,0.962,0.021,0.022,0.021,NaN,NaN,NaN
75,800.0,inf,0.000000e+00,0.974,0.027,0.027,0.027,NaN,NaN,NaN
76,900.0,inf,0.000000e+00,0.978,0.033,0.034,0.033,NaN,NaN,NaN


In [11]:
# pickleFile = 'charginoData.pcl'
pickleFile = 'stauData.pcl'
print('Saving to',pickleFile)
df.to_pickle(pickleFile)

Saving to stauData.pcl


In [12]:
massWidth = set([])
for irow,row in df.iterrows():
    m,w = row['mass (GeV)'],row['lifetime (ns)']
    massWidth.add((m,w))
massWidth = sorted(list(massWidth))
print(len(massWidth))
print(massWidth)

78
[(100.0, 0.3), (100.0, 1.0), (100.0, 3.0), (100.0, 10.0), (100.0, 30.0), (100.0, inf), (200.0, 0.3), (200.0, 1.0), (200.0, 3.0), (200.0, 10.0), (200.0, 30.0), (200.0, inf), (300.0, 0.3), (300.0, 1.0), (300.0, 3.0), (300.0, 10.0), (300.0, 30.0), (300.0, inf), (400.0, 0.3), (400.0, 1.0), (400.0, 3.0), (400.0, 10.0), (400.0, 30.0), (400.0, inf), (450.0, 0.3), (450.0, 1.0), (450.0, 3.0), (450.0, 10.0), (450.0, 30.0), (450.0, inf), (500.0, 0.3), (500.0, 1.0), (500.0, 3.0), (500.0, 10.0), (500.0, 30.0), (500.0, inf), (550.0, 0.3), (550.0, 1.0), (550.0, 3.0), (550.0, 10.0), (550.0, 30.0), (550.0, inf), (600.0, 0.3), (600.0, 1.0), (600.0, 3.0), (600.0, 10.0), (600.0, 30.0), (600.0, inf), (650.0, 0.3), (650.0, 1.0), (650.0, 3.0), (650.0, 10.0), (650.0, 30.0), (650.0, inf), (700.0, 0.3), (700.0, 1.0), (700.0, 3.0), (700.0, 10.0), (700.0, 30.0), (700.0, inf), (800.0, 0.3), (800.0, 1.0), (800.0, 3.0), (800.0, 10.0), (800.0, 30.0), (800.0, inf), (900.0, 0.3), (900.0, 1.0), (900.0, 3.0), (900.0, 

In [14]:
df[df['mass (GeV)'] == 400.0]

,mass (GeV),lifetime (ns),width (GeV),acceptance,event eff,SR-Inclusive_Low eff,SR-Inclusive_High eff,Expected Limit (pb),Observed Limit (pb),Signal Cross-Section (pb)
3,400.0,0.3,2.194000e-15,0.025,0.003,0.012,0.003,NaN,NaN,NaN
16,400.0,1.0,6.582000e-16,0.000,0.000,0.000,0.000,NaN,NaN,NaN
29,400.0,3.0,2.194000e-16,0.662,0.025,0.035,0.025,0.00195,0.00339,0.00186
42,400.0,10.0,6.582000e-17,0.843,0.028,0.031,0.028,0.00132,0.00231,0.00186
55,400.0,30.0,2.194000e-17,0.885,0.019,0.020,0.019,0.00215,0.00380,0.00186
68,400.0,inf,0.000000e+00,0.901,0.004,0.005,0.004,0.01440,0.01790,0.00186
